<a href="https://colab.research.google.com/github/jenny005/Reinforcement-Learning-by-Sutton-Barto/blob/main/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3 : Finite Markov Decision Processes

# Gridworld

In [ ]:
import numpy as np

class GridworldEnv:
    def __init__(self, height=5, width=5):
        self.height = height
        self.width = width
        self.action_space = [0, 1, 2, 3]  # 0: Up, 1: Down, 2: Left, 3: Right
        self.A = (0, 1)
        self.A_prime = (4, 1)
        self.B = (0, 3)
        self.B_prime = (2, 3)

    def step(self, state, action):
        i, j = state

        # Special teleportation states
        if state == self.A:
            return self.A_prime, 10
        elif state == self.B:
            return self.B_prime, 5

        # Action to direction mapping
        directions = {
            0: (-1, 0),  # Up
            1: (1, 0),   # Down
            2: (0, -1),  # Left
            3: (0, 1)    # Right
        }

        di, dj = directions[action]
        ni, nj = i + di, j + dj

        # Check boundary conditions
        if 0 <= ni < self.height and 0 <= nj < self.width:
            return (ni, nj), 0
        else:
            return (i, j), -1

    def get_all_states(self):
        return [(i, j) for i in range(self.height) for j in range(self.width)]

    def get_available_actions(self, state):
        return self.action_space


In [ ]:
env = GridworldEnv()

print("From A:", env.step(env.A, 0))          # Expected: ((4, 1), 10)
print("From B:", env.step(env.B, 1))          # Expected: ((2, 3), 5)
print("From (0, 0) going up:", env.step((0, 0), 0))  # Expected: ((0, 0), -1)
print("From (2, 2) going right:", env.step((2, 2), 3))  # Expected: ((2, 3), 0)


From A: ((4, 1), 10)
From B: ((2, 3), 5)
From (0, 0) going up: ((0, 0), -1)
From (2, 2) going right: ((2, 3), 0)


In [ ]:
import numpy as np

# Define the Gridworld
grid_size = 4
states = [(i, j) for i in range(grid_size) for j in range(grid_size)]
actions = ['U', 'D', 'L', 'R']  # Up, Down, Left, Right
gamma = 1.0
theta = 1e-4  # Convergence threshold

# Transition dynamics
def step(state, action):
    i, j = state
    if state == (0, 0) or state == (3, 3):
        return state, 0  # Terminal state

    if action == 'U':
        i = max(i - 1, 0)
    elif action == 'D':
        i = min(i + 1, grid_size - 1)
    elif action == 'L':
        j = max(j - 1, 0)
    elif action == 'R':
        j = min(j + 1, grid_size - 1)

    return (i, j), -1

# Initialize value function
V = np.zeros((grid_size, grid_size))

# Iterative Policy Evaluation
iteration = 0
while True:
    delta = 0
    for i in range(grid_size):
        for j in range(grid_size):
            state = (i, j)
            if state == (0, 0) or state == (3, 3):
                continue  # Skip terminal states

            v = V[i, j]
            new_v = 0
            for action in actions:
                next_state, reward = step(state, action)
                ni, nj = next_state
                new_v += 0.25 * (reward + gamma * V[ni, nj])  # uniform policy

            V[i, j] = new_v
            delta = max(delta, abs(v - new_v))
    iteration += 1
    if delta < theta:
        break

# Print the final value function
print(f"Converged in {iteration} iterations.\n")
print("Optimal state values under uniform policy:\n")
for row in V:
    print(["{:+.2f}".format(val) for val in row])


Converged in 114 iterations.

Optimal state values under uniform policy:

['+0.00', '-14.00', '-20.00', '-22.00']
['-14.00', '-18.00', '-20.00', '-20.00']
['-20.00', '-20.00', '-18.00', '-14.00']
['-22.00', '-20.00', '-14.00', '+0.00']
